In [ ]:
!nvidia-smi

Sun May 15 17:22:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers datasets rouge_score sentencepiece

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from datasets import load_dataset
from tqdm.notebook import tqdm
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
root = "/content/drive/MyDrive/dataset/ADL2022/HW3/"

In [ ]:
data_files = {"train": root+"data/train.jsonl",
              "test": root+"data/public.jsonl"}
dataset = load_dataset("json", data_files=data_files)

Using custom data configuration default-aa920f60a05dd52d
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-aa920f60a05dd52d/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['date_publish', 'title', 'source_domain', 'maintext', 'split', 'id'],
        num_rows: 21710
    })
    test: Dataset({
        features: ['date_publish', 'title', 'source_domain', 'maintext', 'split', 'id'],
        num_rows: 5494
    })
})

In [ ]:
def show_samples(dataset, num_samples=3, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> text: {example['maintext']}'")
        print(f"'>> summary: {example['title']}'")


show_samples(dataset)

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/json/default-aa920f60a05dd52d/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-dccff5f6408a15e6.arrow



'>> text: 【圖／文：行遍天下．空拍圖片由讀者提供】
每到東海岸，耳裡總是哼唱著胡德夫《太平洋的風》，彷彿太平洋的風真的就會徐徐吹來、就會吹過臉頰，與最親愛的閨蜜相約一起，走一趟吹吹風的小旅行。
遇見最迷你部落的感動瞬間
車行東海岸，總忍不住想搖下車窗吹風，悠悠晃晃穿梭山海之間，來到花蓮最南端的豐濱鄉，首先印入眼簾的是「高山森林基地」。
在高山森林基地的獵人導引之下，終於跟閨蜜登上勇氣石。
「高山」是部落的名字，祖先翻山越嶺從高海拔遷徙到海邊，是全台灣唯一一支靠海的布農族，也是全台最迷你的部落。陽光穿透基地正中央大樹，射出彷彿耶穌光的光芒，部落青年小馬馬中原就在大樹光束、炊煙裊裊中吟唱起部落的歌曲。他們為體驗者吟唱祝福的部落歌曲、說部落遷移的故事；接著在小馬帶領下，體驗攀樹「找老鷹」用鷹的視野俯瞰部落；或感受「找獵人」進入森林獵場，登上高點勇氣石，眺望綿延的綠林和海洋。聽文化的故事也了解大自然傳達給人類的訊息。
找老鷹的體驗，可以感受與自然相親感。
延續著海岸線，不遠處豐富部落的「海岸咖啡莊園」同樣也迷人，以部落共好為前提，頭目許永哲以友善農法栽種咖啡，揉合出一種帶著太平洋風微鹹滋味的咖啡。跟著族人往山裡走在半山腰的咖啡林間一起手烘咖啡，搖晃間仔細聆聽咖啡第一爆出淺焙的訊息，餐點當然就要吃部落媽媽親手做的產地料理，聽自在的吟唱聲，感受最純粹的食物原味。
海岸咖啡莊園的手烘咖啡體驗，感動於手烘的真實感。
有意義的旅行挑戰
吃一次地產地銷食物
這趟旅行的意義，在於地產地銷的食物選擇，高山森林基地與海岸咖啡莊園的餐廳都提供了來自部落、村落的農產品，把食物運送里程降到最低，只選當季、當地食材，也是一種愛地球的旅行方式！
去吹太平洋的風
海岸咖啡莊園
地址：花蓮縣豐濱鄉豐富部落122號
電話：0936-302-719
網址：costal-coffee.com
價位：敬請預約豐富部落咖啡莊園半日遊1人1,880元（10人成行，含部落導覽、午或晚餐、烘豆體驗，需2週前預約）
高山森林基地
地址：花蓮縣豐濱鄉磯崎村高山一鄰2號（往山上）
電話：0933-991-926（敬請預約）
網址：www.gs-forest.com
價位：找獵人體驗1人1,480元（10-20人成行，約半日時間，含講解、射箭文化體驗、獵人餐）、找老鷹體驗1人1,480元（10-20人成行，約半日時間

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_checkpoint = "google/mt5-small"
tokenizer_checkpoint = model_checkpoint#"ckiplab/bert-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


In [ ]:
inputs = tokenizer("我家門前有小河，後面有山坡。")
inputs

{'input_ids': [259, 221108, 27304, 2884, 1637, 2144, 8805, 261, 7605, 4874, 1637, 2619, 105511, 306, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

['▁', '我家', '門', '前', '有', '小', '河', ',', '後', '面', '有', '山', '坡', '。', '</s>']

In [ ]:
max_input_length = 512
max_target_length = 30

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["maintext"], max_length=max_input_length, truncation=True
    )
    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["title"], max_length=max_target_length, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-aa920f60a05dd52d/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-02bad7c8796195b1.arrow


  0%|          | 0/6 [00:00<?, ?ba/s]

In [ ]:

tokenizer("你好")

{'input_ids': [259, 4235, 3586, 1], 'attention_mask': [1, 1, 1, 1]}

In [ ]:
generated_summary = ["我超級愛飢餓遊戲", "我非常想上廁所"]
reference_summary = ["我愛飢餓遊戲","我想上廁所"]

from datasets import load_metric

rouge_score = load_metric("rouge")

scores = rouge_score.compute(
    predictions=[tokenizer(generated_summary).input_ids], references=[tokenizer(reference_summary).input_ids]
)
scores

{'rouge1': AggregateScore(low=Score(precision=0.7058823529411765, recall=0.9230769230769231, fmeasure=0.8000000000000002), mid=Score(precision=0.7058823529411765, recall=0.9230769230769231, fmeasure=0.8000000000000002), high=Score(precision=0.7058823529411765, recall=0.9230769230769231, fmeasure=0.8000000000000002)),
 'rouge2': AggregateScore(low=Score(precision=0.5625, recall=0.75, fmeasure=0.6428571428571429), mid=Score(precision=0.5625, recall=0.75, fmeasure=0.6428571428571429), high=Score(precision=0.5625, recall=0.75, fmeasure=0.6428571428571429)),
 'rougeL': AggregateScore(low=Score(precision=0.7058823529411765, recall=0.9230769230769231, fmeasure=0.8000000000000002), mid=Score(precision=0.7058823529411765, recall=0.9230769230769231, fmeasure=0.8000000000000002), high=Score(precision=0.7058823529411765, recall=0.9230769230769231, fmeasure=0.8000000000000002)),
 'rougeLsum': AggregateScore(low=Score(precision=0.7058823529411765, recall=0.9230769230769231, fmeasure=0.80000000000000

In [ ]:
scores["rougeL"].mid.fmeasure

0.8000000000000002

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at google/mt5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


In [ ]:

from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(
    dataset["train"].column_names
)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['date_publish', 'title', 'source_domain', 'maintext', 'split', 'id'],
        num_rows: 21710
    })
    test: Dataset({
        features: ['date_publish', 'title', 'source_domain', 'maintext', 'split', 'id'],
        num_rows: 5494
    })
})

In [ ]:

tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 21710
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5494
    })
})

In [ ]:
features = [tokenized_datasets["train"][i] for i in range(2)]
data_collator(features)

{'input_ids': <tf.Tensor: shape=(2, 512), dtype=int32, numpy=
array([[   259,  71934,   2144, ...,  24134,  54333,      1],
       [   259, 170885,  29419, ..., 179975,  19495,      1]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int32)>, 'labels': <tf.Tensor: shape=(2, 30), dtype=int32, numpy=
array([[   259,  74378,   9734,  22399,   4457,   1146,  91653,   9139,
        171388,  26473,  31225,    259,    292,   2230, 200229, 192250,
          4066,  23281,   2910,   4565,  86305,  95289, 241245, 178273,
         63687,  18540, 137179,   1193, 175062,      1],
       [  6874, 240277,   1193,    879,   7274,   1637,   8043,   4417,
           291,   1193, 196420,  44654,  20726,  23392,  20794,  11022,
        100430,    259,  74387,  39240,   2884, 141783,  82813,   3801,
         50301,  44932, 103102,      1,   -100,   -100]], dtype=int32)>, 'decoder_input_ids': <tf.Tensor: 

In [ ]:
pt_model.save_pretrained("path/to/awesome-name-you-picked")

In [ ]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=8
    ,
)
tf_eval_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=4,
)

In [ ]:
from transformers import create_optimizer
import tensorflow as tf

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_epochs = 1
num_train_steps = len(tf_train_dataset) * num_train_epochs
model_name = model_checkpoint.split("/")[-1]

optimizer, schedule = create_optimizer(
    init_lr=1e-3,
    num_warmup_steps=10,
    num_train_steps=num_train_steps,
    weight_decay_rate=0,
)

model.compile(optimizer=optimizer)

# Train in mixed-precision float16
# tf.keras.mixed_precision.set_global_policy("mixed_float16")

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour, please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:

# from transformers.keras_callbacks import PushToHubCallback

# callback = PushToHubCallback(
#     output_dir=f"{model_name}-finetuned-amazon-en-es", tokenizer=tokenizer
# )

# model.fit(
#     tf_train_dataset, validation_data=tf_eval_dataset, epochs=8
# )

In [ ]:
def rouge_scores(labels, logits):
  # get labels from probability
  pred_labels = np.argmax(logits, axis=-1)
  rouge_result = rouge_score.compute(predictions=[pred_labels], references=[labels])
  rouge1, rouge2, rougeL, rougeLsum= \
  rouge_result["rouge1"].mid.fmeasure, rouge_result["rouge2"].mid.fmeasure, rouge_result["rougeL"].mid.fmeasure, \
  rouge_result["rougeLsum"].mid.fmeasure
  return {"rouge1":rouge1, "rouge2":rouge2, "rougeL":rougeL, "rougeLsum":rougeLsum}

def rougeL_loss(labels, logits, rouge_type="rougeL"):
  # get labels from probability
  pred_labels = tf.math.argmax(logits, axis=-1)
  rouges = []
  for p, l in zip(pred_labels, labels):
    rouge_result = rouge_score.compute(predictions=[pred_labels], references=[labels])
    rouge = rouge_result[rouge_type].mid.fmeasure
    rouges.append(rouge)
  rouges = np.array(rouges, dtype=float).reshape(-1,1,1)

  return -tf.reduce_mean(tf.math.multiply(tf.math.log_softmax(logits), rouges)) # 16 * 16*30*vocab_size = 16*30*vocab_size

In [ ]:
# @tf.function
def train_step(input_ids, labels, rouge_type):
  with tf.GradientTape() as tape:
    # Run the forward pass of the layer.
    # The operations that the layer applies
    # to its inputs are going to be recorded
    # on the GradientTape.
    outputs = model(input_ids=input_ids, labels=labels)
    celoss, logits = tf.reduce_mean(outputs["loss"]), outputs["logits"] #model(x_batch_train, training=True)  # Logits for this minibatch
    # print(model.predict(input_ids))
    rloss = rougeL_loss(labels, logits, rouge_type)
    loss_value = celoss + rloss

  grads = tape.gradient(loss_value, model.trainable_weights)

  # Run one step of gradient descent by updating
  # the value of the variables to minimize the loss.
  optimizer.apply_gradients(zip(grads, model.trainable_weights))
  return loss_value, celoss

In [ ]:
def update_rouges(original, new):
  objs = ["rouge1","rouge2","rougeL","rougeLsum"]
  for o in objs:
    original[o].append(new[o])
  return

In [ ]:
rouge_type ="rouge2"

In [ ]:
step = 0
logging_steps = 200 # 6000/8

all_loss = []
ce_loss = []
test_rouges = {"epoch":[],"rouge1":[], "rouge2":[], "rougeL":[], "rougeLsum":[]}

for epoch in range(num_train_epochs):
    print("\nStart of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for train in tqdm(tf_train_dataset, total=len(tf_train_dataset)):
        input_ids, labels = train["input_ids"], train["labels"]
        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        loss_value, celoss = train_step(input_ids, labels, rouge_type)
        # loss_value = loss_fn(y_batch_train, logits)
        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        

        # Log every logging_steps batches.
        if step % logging_steps == 0 or step==0 or step==num_train_steps:
            all_loss.append(float(loss_value))
            ce_loss.append(float(celoss))
            print(
                f"Training loss (for one batch) at step {step}: {loss_value}"
            )
            print(
                f"for just cross entropy part, loss: {float(celoss)}"
            )

            print("Seen so far: %s samples" % ((step + 1) * 8))
            # record testing rouge scores
            # temp_rouge = {"rouge1":[], "rouge2":[], "rougeL":[], "rougeLsum":[]}
            # for test in tqdm(tf_eval_dataset, total=len(tf_eval_dataset)):
            #   input_ids, labels = test["input_ids"], test["labels"]
            #   outputs = model(input_ids=input_ids, labels=labels)
            #   logits = outputs["logits"]
            #   update_rouges(temp_rouge, rouge_scores(labels, logits)) #temp_rouge.update(rouge_scores(labels, logits))
            # update_rouges(test_rouges, {k:sum(v)/len(v) for k,v in temp_rouge.items()}) #test_rouges.update({k:sum(v) for k,v in temp_rouge.items()})
            # test_rouges["epoch"].append(step /(num_train_epochs*len(tf_train_dataset)))
            # print(f"Testing rouges: { { k:sum(v)/len(v) for k,v in temp_rouge.items()} }")
        step += 1


Start of epoch 0


  0%|          | 0/2713 [00:00<?, ?it/s]

Training loss (for one batch) at step 0: 19.210838317871094
for just cross entropy part, loss: 18.720874786376953
Seen so far: 8 samples


Exception ignored in: <generator object tqdm.__iter__ at 0x7f6baaf34d50>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tqdm/std.py", line 1196, in __iter__
    yield obj
KeyboardInterrupt: 


KeyboardInterrupt: ignored

In [ ]:
# save results and model
import pickle


train_results = {"total_loss":all_loss, "ce_loss":ce_loss, "test_rouges":test_rouges}
with open(root+"RL/"+rouge_type+"_policy_resultes.pkl", "wb") as f:
  pickle.dump(train_results, f)

model.save_pretrained(root+"RL/"+rouge_type)

In [ ]:
with open(root+"RL/"+rouge_type+"_model.pkl", "wb") as f:
  pickle.dump(model, f)

In [ ]:
with open(root+"RL/"+rouge_type+"_model.pkl", "rb") as f:
  rouge2 = pickle.load(f)

NameError: ignored

In [ ]:
pred_ids = []
for test in tqdm(tf_eval_dataset, total=len(tf_eval_dataset)):
  input_ids, labels = test["input_ids"], test["labels"]
  pred_ids.append(np.argmax(model(input_ids=input_ids, labels=labels)["logits"], axis=-1))
# like [np.array(), np.array(),...]

  0%|          | 0/1374 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
temp = []
for pred_id in tqdm(pred_ids):
  for p in pred_id:
    temp.append(list(p))
pred_ids = temp

In [ ]:
len(pred_ids)

In [ ]:
preds = []
for id in pred_ids:
  preds.append(tokenizer.convert_ids_to_tokens(id))

In [ ]:
preds = ["".join(pred) for pred in preds]

In [ ]:
# preds

In [ ]:
preds = [pred[1:] for pred in preds]
preds = [pred.replace("▁<extra_id_0>", "") for pred in preds]
preds = [pred.replace("</s>", "") for pred in preds]
preds = [pred.replace("▁▁", "") for pred in preds]
preds = [pred.replace("▁", "") for pred in preds]

In [ ]:
# preds

In [ ]:
import json
test_jsonl = []
with open(root+"data/public.jsonl", "r") as file_list:
  for j in file_list:
    test_jsonl.append(json.loads(j))

In [ ]:
test_id = [t["id"] for t in test_jsonl]

In [ ]:
import json
with open(root+"output/RL_test_"+rouge_type+".jsonl", 'w') as f:
  for id, p in zip(test_id, preds):
    f.write(json.dumps({"id":id, "title":p}) + "\n")

rougeL : 
{
  "rouge-1": {
    "r": 0.24346591669294257,
    "p": 0.2881311910753316,
    "f": 0.26189572090120455
  },
  "rouge-2": {
    "r": 0.0866767221620477,
    "p": 0.09591283951209828,
    "f": 0.09036334413900743
  },
  "rouge-l": {
    "r": 0.23317436563826172,
    "p": 0.27547083968807873,
    "f": 0.25062248122568465
  }
}

rouge1:

rouge2:

In [ ]:
rouge_types = ["rouge1", "rouge2", "rougeL"]
